# Условие

https://stepik.org/lesson/573081/step/2 [статья 1](https://habr.com/ru/articles/823644/) [статья 2](https://loginom.ru/blog/quality-metrics)

Вот вам простенькая нейросеть

<img src='https://ucarecdn.com/d40e2fdb-2722-4839-95e8-e90cf679fce5/'>

И тренировочная выборка

| x | y |
|---|---|
| -1 | 1 |
| 0 | 0 |
| 1 | -1 |

Для человека не составит труда восстановить зависимость между $x$ и $y$. Ну да, $y=-x$. То есть оптимальные значения весов сети $w_1=-1,w_0=0$. Посмотрим, сумеет ли нейронная сеть обнаружить эти значения в процессе тренировки.

Для этого нужно составить функцию потерь $L(w)$, и начать процедуру градиентного спуска (будем применять классический градиентный спуск без всяких наворотов). Для старта спуска возьмем
$a_0=(0,0),h=0.1$, причем мы предполагаем, что первая координата позиции "шарика" соответствует весу $w_0$, а вторая координата соответствует весу $w_1$.

Давайте сделаем два шага градиентного спуска и получим позицию $a_2$. В окошко ответа запишите вторую координату (которая соотвтетствует весу $w_1$) позиции $a_2$.

Ну как? Стала ли позиция "шарика" $a_2$ ближе к точке истинной минимума функции потерь $(0,-1)$?

In [ ]:
# -0.64

# Решение

## Функция потерь и частные производные
- $ N $ — количество наблюдений в выборке,
- $ x_i, y_i $ — тренировочные данные,
- $ w_0, w_1 $ — значения весов,
- $y_{pred} $ — предсказания модели

Выход нашей сети $ y_{\text{pred}} $:

$$
y_{\text{pred}} = w_1 \cdot x + w_0
$$

Частные производные, понадобятся:

$$\frac{\partial y_{{pred}}}{\partial w_0} = 1$$

$$\frac{\partial y_{{pred}}}{\partial w_1} = x$$

Функция потерь в задаче не задана, пусть будет MSE (традиционно), однако без нормировки на N (нетрадиционно, но нужно для правильного ответа):

$$
L(w_0, w_1) = \sum_{i=1}^{N} (y_i - y_{pred})^2
$$

Здесь тоже берём производную, сначала от внешней функции. Производная функции потерь $ L $ по $ y_{\text{pred}} $ равна:

$$
\frac{\partial L}{\partial y_{{pred}}} = \frac{\partial}{\partial y_{{pred}}} \left( \sum_{i=1}^{N} \left( y_i - y_{{pred}} \right)^2 \right) = \frac{\partial L}{\partial y_{{pred}}} = -2 \sum_{i=1}^{N} \left( y_i - y_{{pred}} \right) = 2 \sum_{i=1}^{N} \left( y_{{pred}} - y_i \right)
$$

Теперь найдём частные производные $ L(w_0, w_1) $ по весам $ w_0 $ и $ w_1 $ с использованием правила цепочки.

Это чуть сложнее, чем раскрыть скобки, но позволит легче перестроиться при замене функции потерь на более сложную.

$$
\frac{\partial L}{\partial w_0} = \frac{\partial L}{\partial y_{pred}} \cdot \frac{\partial y_{\text{pred}}}{\partial w_0} = 2 \sum_{i=1}^{N} \left(  y_{pred} - y_i \right) \cdot 1
$$

Аналогично:

$$
\frac{\partial L}{\partial w_1} = \frac{\partial L}{\partial y_{\text{pred}}} \cdot \frac{\partial y_{\text{pred}}}{\partial w_1} = 2 \sum_{i=1}^{N} \left(  y_{pred} - y_i \right) \cdot x_i
$$

Подставим $ y_{\text{pred}} = w_1 \cdot x_i + w_0 $ в полученные выражения для частных производных:

$$
\frac{\partial L}{\partial w_0} = 2 \sum_{i=1}^{N} \left( (w_1 \cdot x_i + w_0) - y_i \right)
$$

$$
\frac{\partial L}{\partial w_1} = 2 \sum_{i=1}^{N} \left( (w_1 \cdot x_i + w_0)- y_i \right) \cdot x_i
$$

Раскрываем сумму, подставив заданные  $ x_i, y_i $:

$$
\frac{\partial L}{\partial w_0} = 2  ((w_1 \cdot x_1 + w_0) - y_1) + ((w_1 \cdot x_2 + w_0) - y_2) + ((w_1 \cdot x_3 + w_0) - y_3)
$$

$$
= 2  ((w_1 \cdot (-1) + w_0) - 1) + ((w_1 \cdot 0 + w_0) - 0) + ((w_1 \cdot 1 + w_0) - (-1)) = 6w_0
$$

Аналогично

$$
\frac{\partial L}{\partial w_0} = 2  ((w_1 \cdot x_1 + w_0) - y_1) \cdot x_1 + ((w_1 \cdot x_2 + w_0) - y_2) \cdot x_2 + ((w_1 \cdot x_3 + w_0) - y_3) \cdot x_3
$$

$$
= 2  ((w_1 \cdot (-1) + w_0) - 1) \cdot (-1) + ((w_1 \cdot 0 + w_0) - 0) \cdot 0 + ((w_1 \cdot 1 + w_0) - (-1)) \cdot 1 = 4w_0 + 4
$$

In [ ]:
#@title Проверка нахождения производной
from sympy import symbols, diff

w_0, w_1 = symbols('w_0 w_1')

x = [-1, 0, 1]
y = [1, 0, -1]

y_pred = [w_1 * xi + w_0 for xi in x]
L = sum([(pred - yi)**2 for pred, yi in zip(y_pred, y)])

display(diff(L, w_0))
display(diff(L, w_1))

6*w_0

4*w_1 + 4

## Градиентный спуск
* 1-й шаг градиентного спуска:

$w_{0(1)} = w_{0(0)} - h \cdot \frac{\partial L}{\partial w_0} = w_{0(0)} - h \cdot 6w_{0(0)} = 0$

$w_{1(1)} = w_{1(0)} - h \cdot \frac{\partial L}{\partial w_1} = w_{1(0)} - h \cdot (4w_{1(0)} + 4) = -0.4$

* 2-й шаг градиентного спуска:

$w_{0(2)} = w_{0(1)} - h \cdot \frac{\partial L}{\partial w_0} = w_{0(1)} - h \cdot 6w_{0(1)} = 0$

$w_{1(2)} = w_{1(1)} - h \cdot \frac{\partial L}{\partial w_1} = w_{1(1)} - h \cdot (4w_{1(1)} + 4) = -0.64$

In [ ]:
#@title Проверка нахождения весов на 1-м и 2-м шагах
der_L_w_0 = lambda w_0: 6 * w_0
der_L_w_1 = lambda w_1: 4 * w_1 + 4

w_0 = 0
w_1 = 0
h = 0.1

for i in range(2):
    w_0 -= h * der_L_w_0(w_0)
    w_1 -= h * der_L_w_1(w_1)
    print(f'Шаг {i + 1}: w_0 = {w_0}, w_1 = {w_1}')

Шаг 1: w_0 = 0.0, w_1 = -0.4
Шаг 2: w_0 = 0.0, w_1 = -0.64


In [ ]:
#@title  Общее решение с возможностью сравнить варианты функции потерь
import numpy as np

x = np.array([-1, 0, 1])
y = np.array([1, 0, -1])

def gradients(w0, w1, x, y):
    N = len(x)

    # dw0 = -2 / N * np.sum(y - (w1 * x + w0)) # версия с нормировкой на N
    # dw1 = -2 / N * np.sum(x * (y - (w1 * x + w0)))

    dw0 = -2 * np.sum(y - (w1 * x + w0))    # версия без нормировки
    dw1 = -2 * np.sum(x * (y - (w1 * x + w0)))

    return dw0, dw1

# Функция для выполнения градиентного спуска
def gradient_descent(w0, w1, x, y, h, steps):
    history = [(w0, w1)]  # сохраняем историю весов для каждого шага
    for step in range(steps):
        dw0, dw1 = gradients(w0, w1, x, y)  # вычисляем градиенты
        w0 -= h * dw0  # обновляем веса
        w1 -= h * dw1
        history.append((w0, w1))  # сохраняем веса после каждого шага
    return w0, w1, history

# Градиентный спуск на 2 шага с шагом h=0.1
steps = 2
w0, w1, weights_history = gradient_descent(w0=0, w1=0, x=x, y=y, h=0.1, steps=steps)

print(f"w0 = {w0}, w1 = {w1}")
print("История весов:")
for step, (w0_step, w1_step) in enumerate(weights_history):
    print(f"Шаг {step}: w0 = {w0_step}, w1 = {w1_step}")


# Vadim Kopeykin Решение и Пояснение

In [ ]:
# Vadim Kopeykin https://stepik.org/lesson/573081/step/2?discussion=8496581&thread=solutions&unit=567630
# пояснение https://ucarecdn.com/6b731c42-9306-49ef-bb3d-e845c8116911/
import numpy as np

w = np.array([0., 0.])
X = np.array([[1., -1.],
              [1.,  0.],
              [1.,  1.]])
y = np.array([1., 0., -1.])

h = 0.1
for i in range(2):
    w -= 2 * (X @ w - y) @ X * h
    print(w[1])

-0.4
-0.64


Возможны неточности, смотри [рисунок](https://ucarecdn.com/6b731c42-9306-49ef-bb3d-e845c8116911/)

У нас есть простой нейрон.

Функция нейрона для одного $ k $-ого примера имеет вид:

$$
y^{(k)}_{pred} = w_0 + x^{(k)} * w_1
$$

где $ y^{(k)}_{pred} $ — это предсказанное значение нейрона на $ k $-ом примере.

Для упрощения выражения добавим единичный вход $ x^{(k)}_0 $, он всегда равен 1:

$$
y^{(k)}_{pred} = 1 * w_0 + x^{(k)}_1 * w_1 = x^{(k)}_0 * w_0 + x^{(k)}_1 * w_1
$$

Запишем функцию нейрона в общем виде для одного $ k $-ого примера:

$$
y^{(k)}_{pred} = \sum_{i} x^{(k)}_i * w_i \tag{1}
$$

где $ i $ — индекс входа и индекс соответствующего веса, $ k $ — индекс примера.

Если приглядеться, это выражение можно записать через скалярное произведение.

То есть, происходит как бы сворачивание выражения по индексу $ i $.

Знак суммы и индекс $ i $ исчезают. А элементы с индексом $ i $ увеличивают свою размерность на 1: числа превращаются в векторы, векторы в матрицы и т. д.:

$$
y^{(k)}_{pred} = \overrightarrow{x}^{(k)} * \overrightarrow{w} \tag{2}
$$

Легким движением руки число $ y^{(k)}_{pred} $ превращается в вектор: $ \overrightarrow{y}_{pred} $,

а вектор $ \overrightarrow{x} $ превращается в матрицу $[X]$:

$$
\overrightarrow{y}_{pred} = [X] * \overrightarrow{w} \tag{2}
$$

_______________________________________

Давайте найдем производную выхода нейрона по весу, смотрим на выражение (1):

$$
\frac{\partial y^{(k)}_{pred}}{\partial w_i} = x^{(k)}_i \tag{3}
$$

Теперь запишем функцию потерь для одного $ k $-ого примера:

$$
L^{(k)} = (y^{(k)}_{pred} - y^{(k)})^2\
$$

где $ y^{(k)}_{pred} $ — это предсказанное значение нейрона на $ k $-ом примере, а $ y^{(k)} $ — значение $ k $-ого целевого признака.

Давайте найдем производную функции потерь по выходу нейрона:

$$
\frac{\partial L^{(k)}}{\partial y^{(k)}_{pred}} = 2 * (y^{(k)}_{pred} - y^{(k)}) \tag{4}
$$

Теперь запишем функцию потерь на всех примерах:

$$
L = \sum_{k} L^{(k)}
$$

А теперь давайте найдем частную производную функции потерь на всех примерах по $ i $-му весу.

Обратите внимание, что функция потерь — сложная функция:

$$
L = L(y_{pred}(w_i))
$$

Поэтому применим правило дифференцирования сложной функции:

$$
\frac{\partial L}{\partial w_i} = \frac{\partial}{\partial w_i}\Big[\sum_{k}L^{(k)}\Big] = \sum_{k} \frac{\partial L^{(k)}}{\partial w_i} = \sum_{k}\frac{\partial L}{\partial y_{pred}^k} * \frac{\partial y^k_{pred}}{\partial w_i}
$$

То, что я выделил цветом (две дроби под знаком суммы), заменим на выражения (3) и (4) соответственно:

$$
\frac{\partial L}{\partial w_i} = 2 * \sum_{k} (y^{(k)}_{pred} - y^{(k)}) * x^{(k)}_i
$$
________________________________

Теперь посмотрите внимательно.

Это выражение можно записать как скалярное произведение вектора $ \overrightarrow{y}_{pred} - \overrightarrow{y} $ на вектор $ \overrightarrow{x}_i $.

То есть, происходит сворачивание выражения по индексу $ k $:

$$
\frac{\partial L}{\partial w_i} = 2 * (\overrightarrow{y}_{pred} - \overrightarrow{y}) \odot x_i
$$

А теперь запишем градиент функции потерь **по вектору** весов.

$$
\frac{\partial L}{\partial w} = 2 * (y_{pred} - y) * [X]^T
$$

Легким движением руки частная производная $ \frac{\partial L}{\partial w_i} $ превращается в производную по вектору $ \frac{\partial L}{\partial \overrightarrow{w}} $, а вектор $ \overrightarrow{x}_i $ превращается в матрицу $[X]$:

$$
\frac{\partial L}{\partial \overrightarrow{w}} = 2 * (\overrightarrow{y}_{pred} - \overrightarrow{y}) \odot [X]
$$

Наконец, когда мы научились находить градиент функции потерь **по вектору** весов **на всех** примерах, запишем одну итерацию градиентного спуска:

$$
\overrightarrow{w}_{n+1} = \overrightarrow{w}_n - \frac{\partial L}{\partial \overrightarrow{w}} * h = \overrightarrow{w}_n - 2 (\overrightarrow{y}_{pred} - \overrightarrow{y} \odot [X] * h)
$$


То, что я выделил цветом, заменим на выражение (2), в итоге получаем:

$$
\overrightarrow{w}_{n+1} = \overrightarrow{w}_n - 2 * ([X] \odot \overrightarrow{w} - \overrightarrow{y}) \odot [X]
$$

Собственно, это выражение и применяется в коде:

```python
for i in range(2):
    w = 2 * ([X] * w - y) * [X]^T
```

# Форум решений

In [ ]:
# Игорь Владимирович Лапшин

derivative = [lambda x: 6 * x, lambda x: 4 * x + 4] # Производные по w0 и по w1
a = [0, 0]
number_of_iterations = 2 #Число шагов
h = 0.1
for n in range(number_of_iterations):
    a[0] -= h * derivative[0](a[0])
    a[1] -= h * derivative[1](a[1])
    print('Шаг: {}    w0 = {}    w1 = {}'.format(1 + n, round(a[0], 3), round(a[1], 3)))

Шаг: 1    w0 = 0.0    w1 = -0.4
Шаг: 2    w0 = 0.0    w1 = -0.64


In [ ]:
# Юрий Выборных https://stepik.org/lesson/573081/step/2?discussion=9942318&thread=solutions&unit=567630

import numpy as np
from sympy import symbols, diff, lambdify
import matplotlib.pyplot as plt


w1, w0 = symbols('w1 w0')

# Запишем нашу функцию
function = (-w1 + w0 - 1) ** 2 + w0 ** 2 + (w1 + w0 + 1) ** 2

# Определим производные нашей функции
df_w0 = diff(function, w0)
display(df_w0)
df_w1 = diff(function, w1)
display(df_w1)

# Зададим первую точку и шаг
a0 = (0, 0)    # w0, w1
h = 0.1

# Находим координаты градиентного спуска
point = a0
points = (a0,)
for _ in range(3):
    ldf_w0 = lambdify(w0, expr=df_w0)(w0=point[0])
    ldf_w1 = lambdify(w1, expr=df_w1)(w1=point[1])
    point = point[0] - h * ldf_w0, point[1] - h * ldf_w1
    points += point,


print(*points, sep='\n')

6*w0

4*w1 + 4

(0, 0)
(0.0, -0.4)
(0.0, -0.64)
(0.0, -0.784)


In [ ]:
#Linear regression
import sympy as sp

x_tr_set = [-1, 0, 1]
y_tr_set = [1, 0, -1]
local_loss = []

a = [0, 0]
learning_rate = 0.1
epochs = 2

w1, w0 = sp.symbols('w1, w0', real=True)

for x_index, x_value in enumerate(x_tr_set):
    local_loss.append((w1 * x_value + w0 - y_tr_set[x_index]) ** 2)

loss = sum(local_loss)
w1_derivative = loss.diff(w1)
w0_derivative = loss.diff(w0)

for epoch in range(epochs):
    a[0] -= learning_rate * w1_derivative.evalf(subs={w1: a[0]})
    a[1] -=  learning_rate * w0_derivative.evalf(subs={w0: a[1]})
    print(f"Epoch: {epoch +1}\t Weight w1: {round(a[0], 3)}\t Weight w0: {round(a[1], 3)}")

6*w0

In [ ]:
import sympy as sp
w0, w1 = sp.symbols('w0 w1')
L = (-w1+w0-1)**2 + w0**2 + (w1 + w0 + 1)**2

dLdw1 = sp.utilities.lambdify(
    (w0, w1),
    sp.diff(L, w1),
)
dLdw0 = sp.utilities.lambdify(
    (w0, w1),
    sp.diff(L, w0),
)

h = 0.1
a_w0 = 0
a_w1 = 0
n = 10

for i in range(0, n + 1):
    print(f'a{i}: ({a_w0},  ({a_w1}))')
    ai_w0 = a_w0 - h * dLdw0(a_w0, a_w1)
    ai_w1 = a_w1 - h * dLdw1(a_w0, a_w1)
    a_w0 = ai_w0
    a_w1 = ai_w1

a0: (0,  (0))
a1: (0.0,  (-0.4))
a2: (0.0,  (-0.64))
a3: (0.0,  (-0.784))
a4: (0.0,  (-0.8704000000000001))
a5: (0.0,  (-0.9222400000000001))
a6: (0.0,  (-0.9533440000000001))
a7: (0.0,  (-0.9720064))
a8: (0.0,  (-0.98320384))
a9: (0.0,  (-0.989922304))
a10: (0.0,  (-0.9939533824))


# Open AI

Давайте начнем с того, что составим функцию потерь для нашей нейронной сети и тренировочной выборки.

### Функция потерь

Пусть выход нейронной сети $ y_{\text{pred}} $ задается как:

$$
y_{\text{pred}} = w_1 \cdot x + w_0
$$

Функция потерь (например, среднеквадратичная ошибка — MSE) на выборке будет выглядеть так:

$$
L(w_0, w_1) = \frac{1}{N} \sum_{i=1}^{N} (y_i - (w_1 \cdot x_i + w_0))^2
$$

Где:
- $ N $ — количество наблюдений в выборке,
- $ x_i, y_i $ — тренировочные данные,
- $ w_0, w_1 $ — параметры модели.

Теперь выразим градиенты функции потерь по каждому из весов $ w_0 $ и $ w_1 $, чтобы использовать их для градиентного спуска:

$$
\frac{\partial L}{\partial w_0} = -\frac{2}{N} \sum_{i=1}^{N} (y_i - (w_1 \cdot x_i + w_0))
$$

$$
\frac{\partial L}{\partial w_1} = -\frac{2}{N} \sum_{i=1}^{N} x_i \cdot (y_i - (w_1 \cdot x_i + w_0))
$$

Затем, используя градиентный спуск, обновим веса на каждом шаге:

$$
w_0^{(t+1)} = w_0^{(t)} - h \cdot \frac{\partial L}{\partial w_0}
$$
$$
w_1^{(t+1)} = w_1^{(t)} - h \cdot \frac{\partial L}{\partial w_1}
$$

Начнем с $ w_0 = 0 $, $ w_1 = 0 $, шаг $ h = 0.1 $ и выполним два шага градиентного спуска. Сейчас я посчитаю это.

После двух шагов градиентного спуска значения весов стали:

$$
w_0 = 0.0, \quad w_1 \approx -0.25
$$

Вес $ w_1 $ начал приближаться к правильному значению $ -1 $, хотя пока значительно меньше по модулю. Параметр $ w_0 $ остался равен нулю, что соответствует точному значению для данной линейной зависимости.


In [ ]:
import numpy as np

x = np.array([-1, 0, 1])
y = np.array([1, 0, -1])

def gradients(w0, w1, x, y):
    N = len(x)

    # dw0 = -2 / N * np.sum(y - (w1 * x + w0)) # версия с нормировкой на N
    # dw1 = -2 / N * np.sum(x * (y - (w1 * x + w0)))

    dw0 = -2 * np.sum(y - (w1 * x + w0))    # версия без нормировки
    dw1 = -2 * np.sum(x * (y - (w1 * x + w0)))

    return dw0, dw1

# Функция для выполнения градиентного спуска
def gradient_descent(w0, w1, x, y, h, steps):
    history = [(w0, w1)]  # сохраняем историю весов для каждого шага
    for step in range(steps):
        dw0, dw1 = gradients(w0, w1, x, y)  # вычисляем градиенты
        w0 -= h * dw0  # обновляем веса
        w1 -= h * dw1
        history.append((w0, w1))  # сохраняем веса после каждого шага
    return w0, w1, history

# Градиентный спуск на 2 шага с шагом h=0.1
steps = 2
w0, w1, weights_history = gradient_descent(w0=0, w1=0, x=x, y=y, h=0.1, steps=steps)

print(f"w0 = {w0}, w1 = {w1}")
print("История весов:")
for step, (w0_step, w1_step) in enumerate(weights_history):
    print(f"Шаг {step}: w0 = {w0_step}, w1 = {w1_step}")


w0 = 0.72, w1 = 0.8799999999999999
История весов:
Шаг 0: w0 = 0, w1 = 0
Шаг 1: w0 = 1.2000000000000002, w1 = 1.6
Шаг 2: w0 = 0.72, w1 = 0.8799999999999999


In [ ]:
import numpy as np

# Данные
x = np.array([-1, 0, 1])
y = np.array([1, 0, -1])

# Градиенты функции потерь
def gradients(w0, w1, x, y):
    N = len(x)
    predictions = w1 * x + w0
    errors = y - predictions
    dw0 = -2 * np.sum(errors) / N  # Градиент по w0
    dw1 = -2 * np.sum(errors * x) / N  # Градиент по w1
    return dw0, dw1

# Функция для выполнения градиентного спуска
def gradient_descent(w0, w1, x, y, h, steps):
    history = [(w0, w1)]  # сохраняем историю весов для каждого шага
    for step in range(steps):
        dw0, dw1 = gradients(w0, w1, x, y)  # вычисляем градиенты
        w0 -= h * dw0  # обновляем w0
        w1 -= h * dw1  # обновляем w1
        history.append((w0, w1))  # сохраняем веса после каждого шага
    return w0, w1, history

# Выполним 2 шага градиентного спуска с шагом h=0.1
steps = 2
w0, w1, weights_history = gradient_descent(w0=0, w1=0, x=x, y=y, h=0.1, steps=steps)

# Вывод результатов
print(f"w0 = {w0}, w1 = {w1}")
print("История весов:")
for step, (w0_step, w1_step) in enumerate(weights_history):
    print(f"Шаг {step}: w0 = {w0_step}, w1 = {w1_step}")


w0 = 0.0, w1 = -0.2488888888888889
История весов:
Шаг 0: w0 = 0, w1 = 0
Шаг 1: w0 = 0.0, w1 = -0.13333333333333333
Шаг 2: w0 = 0.0, w1 = -0.2488888888888889


In [ ]:
import numpy as np

# Тренировочная выборка
x_tr_set = [-1, 0, 1]
y_tr_set = [1, 0, -1]

# Параметры обучения
learning_rate = 0.1
epochs = 2

# Начальные веса
w1 = 0.0  # Наклон
w0 = 0.0  # Смещение

# Градиенты функции потерь
def compute_gradients(w0, w1, x, y):
    N = len(x)
    dw0 = 0.0
    dw1 = 0.0

    # Вычисляем градиенты
    for i in range(N):
        prediction = w1 * x[i] + w0
        error = prediction - y[i]
        dw0 += error  # Градиент по w0
        dw1 += error * x[i]  # Градиент по w1

    dw0 = (2 / N) * dw0  # Средний градиент по w0
    dw1 = (2 / N) * dw1  # Средний градиент по w1

    return dw0, dw1

# Обучение
for epoch in range(epochs):
    dw0, dw1 = compute_gradients(w0, w1, x_tr_set, y_tr_set)  # Вычисляем градиенты
    w0 -= learning_rate * dw0  # Обновляем w0
    w1 -= learning_rate * dw1  # Обновляем w1

    print(f"Epoch: {epoch + 1}\t Weight w1: {round(w1, 3)}\t Weight w0: {round(w0, 3)}")

# Вывод итоговых значений весов
print(f"Final Weights: w0 = {round(w0, 3)}, w1 = {round(w1, 3)}")


Epoch: 1	 Weight w1: -0.133	 Weight w0: 0.0
Epoch: 2	 Weight w1: -0.249	 Weight w0: 0.0
Final Weights: w0 = 0.0, w1 = -0.249
